In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import riskfolio as rp
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
#Import selected ticker for portfolio selection
tickers = ['XDW0.MI','XDWT.MI','XDWC.MI','XDWH.MI','NUCL.MI','INFR.MI','XDWM.MI','XAIX.MI','ENER.MI','SEME.MI']
tickers.sort()

start = "2019-01-01"
end = datetime.today().date()

#Download Time series and calculate features
P = yf.download(tickers, start=start, end=end)["Close"]
P.index = pd.to_datetime(P.index)
P.columns = tickers


P2 = P[['XDWH.MI','XDWT.MI','INFR.MI','XDWM.MI']]
P2.bfill
sample = pd.DataFrame(P2.columns)

Y = P2.pct_change().dropna() 
display(Y.head())
display(Y.tail())

dcorr = pd.DataFrame(np.corrcoef(Y))
corr = Y.corr()
display(corr)
# Creare portafoglio ipotizzando i 4 ETF selezionati prima, utilizzando il metodo minRisk e comparando
# le diverse misure di rischio prima (optare per il CVaR poi con estimation mean BS), mettere eventuali
# constraints se la minimizzazione del rischio non restituisce un pesi ottimali
# Create the portfolio


In [ ]:
# Creo oggetto portafoglio
port = rp.Portfolio(returns=Y)
method_mu = "JS"
method_cov = "ledoit"

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d = 0.94)
model = "Classic"
obj = "Sharpe"
rf = 0
l = 0

# Creo portafoglio con le misure di rischio da minimizzare diverse e confronto graficamente i pesi
rms = ['MV', 'MAD', 'MSV', 'FLPM', 'SLPM', 'CVaR',
        'WR', 'MDD', 'ADD', 'CDaR', 'UCI']

w_s = pd.DataFrame([])

for i in rms:
    w = port.optimization(model=model, rm=i, obj=obj, rf=rf, l=l, hist=True)
    w_s = pd.concat([w_s, w], axis=1)
    
w_s.columns = rms

w_s.style.format("{:.2%}").background_gradient(cmap='YlGn')

In [ ]:
#Seleziono le metriche di rischio che voglio approfondire
rms2 = ["MV","ADD","CVaR", "CDaR"]

w_s2 = pd.DataFrame([])

for i in rms2:
    w2 = port.optimization(model=model, rm=i, obj=obj, rf=rf, l=l, hist=True)
    w_s2 = pd.concat([w_s2, w2], axis=1)
    
w_s2.columns = rms2
w_s2.style.format("{:.2%}").background_gradient(cmap='YlGn')


In [ ]:
#Create singulare Pie chart

for i in w_s2.columns:
    ax = rp.plot_pie(w=w_s2[[i]], title=f'Portfolio with {i} Risk Measure', others=0.05, nrow=25, cmap = "tab20",
                     height=6, width=10, ax=None)
    plt.show()

# Plottare il grafico dei rendimenti cumulati
ax1 = rp.plot_series(returns=Y,
                    w = w_s2,
                    cmap="tab20",
                    height=6,
                    width=10,
                    ax=None)


In [ ]:
# Inserisco i constraints
asset_classes = {'Assets': ['XDWH.MI','XDWT.MI','INFR.MI','XDWM.MI'], 
                 'Industry': ['Health Care','Information Technology',
                              'Infrastructure','Materials']}

asset_classes = pd.DataFrame(asset_classes)
asset_classes = asset_classes.sort_values(by=['Assets'])

constraints = {'Disabled': [False, False, False, False, False],
               'Type': ['All Assets', 'Classes', 'Classes', 'Classes',
                        'Classes'],
               'Set': ['', 'Industry', 'Industry', 'Industry', 'Industry'],
               'Position': ['', 'Health Care', 'Information Technology', 'Infrastructure',
                            'Materials'],
               'Sign': ['>=', '>', '>', '>', '>'],
               'Weight': [0.1, 0, 0, 0, 0],
               'Type Relative': ['', '', '', '', ''],
               'Relative Set': ['', '', '', '', ''],
               'Relative': ['', '', '', '', ''],
               'Factor': ['', '', '', '', '']}

constraints = pd.DataFrame(constraints)
display(constraints)
A, B = rp.assets_constraints(constraints, asset_classes)

port2 = rp.Portfolio(returns=Y)
port2.assets_stats(method_mu=method_mu, method_cov=method_cov, d = 0.94)
port2.ainequality = A
port2.binequality = B

w_s3 = pd.DataFrame([])

for i in rms2:
    w3 = port2.optimization(model=model, rm=i, obj=obj, rf=rf, l=l, hist=True)
    w_s3 = pd.concat([w_s3, w3], axis=1)

w_s3.columns = rms2
w_s3.style.format("{:.2%}").background_gradient(cmap='YlGn')


In [ ]:
#Create singulare Pie chart

for i in w_s3.columns:
    ax = rp.plot_pie(w=w_s3[[i]], title=f'Portfolio with {i} Risk Measure', others=0.05, nrow=25, cmap = "tab20",
                     height=6, width=10, ax=None)
    plt.show()

# Plottare il grafico dei rendimenti cumulati
ax1 = rp.plot_series(returns=Y,
                    w = w_s3,
                    cmap="tab20",
                    height=6,
                    width=10,
                    ax=None)

In [ ]:
# Creare le frontiere efficienti per ogni metodologia di risk measure
#Mean - Variance
points = 20 # Number of points of the frontier

frontierMV = port2.efficient_frontier(model=model, rm="MV", points=points, rf=rf, hist=True)

display(frontierMV)

# Grafico
label = 'Max Sharpe Ratio' # Title of point

axMV = rp.plot_frontier(w_frontier=frontierMV, mu=port2.mu, cov=port2.cov, returns=Y, rm="MV",
                      rf=rf, alpha=0.05, cmap='viridis', w=w_s2[["MV"]], label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)
plt.show()

axRet = rp.plot_series(returns=Y, w=frontierMV, cmap="tab20", height=6, width=10, ax=None)
plt.show()


In [ ]:
#Average Drawdown
points = 20 # Number of points of the frontier

frontierADD = port2.efficient_frontier(model=model, rm="ADD", points=points, rf=rf, hist=True)

display(frontierADD)

# Grafico
label = 'Max Sharpe Ratio' # Title of point

axADD = rp.plot_frontier(w_frontier=frontierADD, mu=port2.mu, cov=port2.cov, returns=Y, rm="ADD",
                      rf=rf, alpha=0.05, cmap='viridis', w=w_s3[["ADD"]], label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)

plt.show()

axRet = rp.plot_series(returns=Y, w=frontierADD, cmap="tab20", height=6, width=10, ax=None)
plt.show()


In [ ]:
#Conditional Valuea at Risk
points = 20 # Number of points of the frontier

frontierCVaR = port2.efficient_frontier(model=model, rm="CVaR", points=points, rf=rf, hist=True)

display(frontierCVaR)

# Grafico
label = 'Max Sharpe Ratio' # Title of point

axCVaR = rp.plot_frontier(w_frontier=frontierCVaR, mu=port2.mu, cov=port2.cov, returns=Y, rm="CVaR",
                      rf=rf, alpha=0.05, cmap='viridis', w=w_s3[["CVaR"]], label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)
plt.show()

axRet = rp.plot_series(returns=Y, w=frontierCVaR, cmap="tab20", height=6, width=10, ax=None)
plt.show()


In [ ]:
#Conditional Drawdown at Risk
points = 20 # Number of points of the frontier

frontierCDaR = port2.efficient_frontier(model=model, rm="CDaR", points=points, rf=rf, hist=True)

display(frontierCDaR)

# Grafico
label = 'Max Sharpe Ratio' # Title of point

axCDaR = rp.plot_frontier(w_frontier=frontierCDaR, mu=port2.mu, cov=port2.cov, returns=Y, rm="CDaR",
                      rf=rf, alpha=0.05, cmap='viridis', w=w_s3[["CDaR"]], label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)
plt.show()

axRet = rp.plot_series(returns=Y, w=frontierCDaR, cmap="tab20", height=6, width=10, ax=None)
plt.show()


In [ ]:
#Confronto Cum Ret dei portafoglio ipoteticamente migliori
w_opt = pd.concat([frontierMV[19],frontierADD[3],frontierCVaR[19],frontierCDaR[9]],axis=1)
w_opt.columns = rms2
axRet = rp.plot_series(returns=Y, w=w_opt, cmap="tab20", height=6, width=10, ax=None)
plt.show()

# risk contribution for each asset in each risk measure considered for the optimal portfolio
rc_MV = rp.Risk_Contribution(w=frontierMV[19], cov=port2.cov, returns=Y, rm="MV", rf=rf)
print("Risk Contribution for each asset with MV method")
display(pd.concat([sample,pd.DataFrame(rc_MV)],axis=1))

rc_ADD = rp.Risk_Contribution(w=frontierADD[3], cov=port2.cov, returns=Y, rm="ADD", rf=rf)
print("\nRisk Contribution for each asset with ADD method")
display(pd.concat([sample,pd.DataFrame(rc_ADD)],axis=1))

rc_CVaR = rp.Risk_Contribution(w=frontierCVaR[19], cov=port2.cov, returns=Y, rm="CVaR", rf=rf)
print("\nRisk Contribution for each asset with CVaR method")
display(pd.concat([sample,pd.DataFrame(rc_CVaR)],axis=1))

rc_CDaR = rp.Risk_Contribution(w=frontierCDaR[9], cov=port2.cov, returns=Y, rm="CDaR", rf=rf)
print("\nRisk Contribution for each asset with CDaR method")
display(pd.concat([sample,pd.DataFrame(rc_CDaR)],axis=1))


# Bar Plot of risk contribution
axMV = rp.PlotFunctions.plot_risk_con(w=pd.DataFrame(frontierMV[19]), cov=port2.cov, returns=Y, rm='MV',
                                             rf=rf, alpha=0.05, height=6, width=10, ax=None)
plt.show()

axADD = rp.PlotFunctions.plot_risk_con(w=pd.DataFrame(frontierADD[3]), cov=port2.cov, returns=Y, rm='ADD',
                                             rf=rf, alpha=0.05, height=6, width=10, ax=None)
plt.show()

axCVaR = rp.PlotFunctions.plot_risk_con(w=pd.DataFrame(frontierCVaR[19]), cov=port2.cov, returns=Y, rm='CVaR',
                                             rf=rf, alpha=0.05, height=6, width=10, ax=None)
plt.show()

axCDaR = rp.PlotFunctions.plot_risk_con(w=pd.DataFrame(frontierCDaR[9]), cov=port2.cov, returns=Y, rm='CDaR',
                                             rf=rf, alpha=0.05, height=6, width=10, ax=None)
plt.show()

In [ ]:
#Create Reports

rp.excel_report(Y,
                frontierMV,
                rf=0,
                alpha=0.05,
                t_factor=252,
                ini_days=1,
                days_per_year=252,
                name="Mean-MV Min Risk")

rp.excel_report(Y,
                frontierADD,
                rf=0,
                alpha=0.05,
                t_factor=252,
                ini_days=1,
                days_per_year=252,
                name="Mean-ADD Min Risk")

rp.excel_report(Y,
                frontierCVaR,
                rf=0,
                alpha=0.05,
                t_factor=252,
                ini_days=1,
                days_per_year=252,
                name="Mean-CVaR Min Risk")

rp.excel_report(Y,
                frontierCDaR,
                rf=0,
                alpha=0.05,
                t_factor=252,
                ini_days=1,
                days_per_year=252,
                name="Mean-CDaR Min Risk")